In [9]:
import os
import shutil
from sklearn.model_selection import train_test_split
import cv2 as cv
import numpy as np

In [10]:
def split_data(source_dir, train_dir, test_dir, test_size=0.2, random_state=None):
    # Sikre, at destinationsmapperne eksisterer
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Hent alle filnavne fra data-mappen
    all_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

    # Opdel filerne i train og test ved hjælp af train_test_split
    train_files, test_files = train_test_split(all_files, test_size=test_size, random_state=random_state)

    # Kopier train filer
    for f in train_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(train_dir, f))
    
    # Kopier test filer
    for f in test_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(test_dir, f))

In [11]:
def get_tiles(image):
    # laver en tom liste 
    tiles = []
    # kører et for loop hvor elementer vil blive tilføjet til listen, hvor y repræsenterer en række af billedet
    for y in range(5):
        tiles.append([])
    # kører et nested loop, hvor billedet bliver delt op i en tavel med 25 kvadrater af 100,100 px og tilføjer til listen tiles
        for x in range(5):
            tiles[-1].append(image[y*100:(y+1)*100, x*100:(x+1)*100])
    return tiles

def save_tile(tile, output_folder, image_name, x, y):
    # her laver vi en mappe 'blandet' hvis den ikke findes
    if not os.path.exists(os.path.join(output_folder, "blandet")):
        os.makedirs(os.path.join(output_folder, "blandet"))

    # her definerer vi navnet på det tile der skal gemmes som f.eks. 1_3_2.png ved brug af en f-streng
    tile_filename = f"{image_name}_{x}_{y}.png"

    # her definerer bi tile_path som er stedet hvor vi vil gemme vores tile
    tile_path = os.path.join(output_folder, "blandet", tile_filename)
    # her gemmer vi vores tile som tile_filename i folderen 'blandet'
    cv.imwrite(tile_path, tile)

    # her skriver vi til konsollen at vi har gemt vores til i 'blandet' folderen
    print(f"Saved Tile as {tile_filename} in 'blandet' folder")

In [12]:
def tile_set_create(output_folder, input_folder):
    # foldere til billeder defineres
    output_folder = "Data/KD train tiles"
    input_folder = "Data/KD train plader"

    # vi kigger igennem hver fil i input folderen
    for filename in os.listdir(input_folder):
        
        if not filename.endswith('.jpg'):
            continue
        # da billederne er navngivet som 1.jpg, 2.jpg osv. 
        # bruger vi filename counteren til at definere billedet vi vil have
        image_path = input_folder +"/" +filename
        # vi udskriver stien til billedet som man kan kigge på hvis der skulle opstå fejl
        print(image_path)
        # her tjekker vi om billedet findes i mappen og skriver "Image not found" hvis det ikke eksisterer
        if not os.path.isfile(image_path):
            print("Image not found")
            return
        
        # her bruger vi openCV til at åbne billedet
        image = cv.imread(image_path)
        tiles = get_tiles(image)
        print(len(tiles))
        for y, row in enumerate(tiles):
            for x, tile in enumerate(row):
                save_tile(tile, output_folder, filename, x, y)



In [13]:
# Stier til mapperne
data_dir = 'Data/KD hele plader'
train_dir = 'Data/KD train plader'
test_dir = 'Data/KD test plader'
tile_train_dir = "Data/KD train tiles"

In [14]:
split_data(data_dir, train_dir, test_dir, test_size=0.2, random_state=42)

En uventet fejl opstod: Random.shuffle() got an unexpected keyword argument 'seed'


In [15]:
tile_set_create(tile_train_dir, train_dir)